In [1]:
! pip install transformers 
! pip install datasets 
! pip install huggingface_hub

In [2]:
#from huggingface_hub import notebook_login

#notebook_login()

In [3]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [4]:
import transformers
print(transformers.__version__)

4.16.2


In [5]:
model_checkpoint = "bert-base-multilingual-cased"
batch_size = 16

In [6]:
#!apt install git-lfs
#!git config --global user.email "patelkrinal214@gmail.com"
#!git config --global user.name "krinal214"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 75 not upgraded.
Need to get 3316 kB of archives.
After this operation, 11.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 git-lfs amd64 2.9.2-1 [3316 kB]
Fetched 3316 kB in 0s (11.2 MB/s)

78Selecting previously unselected package git-lfs.
(Reading database ... 103274 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.9.2-1_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [###########...............................................] 8Unpacking git-lfs (2.9.2-1) ...
7Progress: [ 40%] [#######################...................................] 8Setting up git-lfs (2.9.2-1) ...
7Progress: [ 60%] [##################################..........

### Loading the dataset

In [7]:
from datasets import load_dataset, load_metric

In [8]:
datasets=load_dataset("tydiqa","secondary_task")
datasets

Downloading:   0%|          | 0.00/3.49k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/161M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset tydiqa downloaded and prepared to /root/.cache/huggingface/datasets/tydiqa/secondary_task/1.0.0/b8a6c4c0db10bf5703d7b36645e5dbae821b8c0e902dac9daeecd459a8337148. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 49881
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5077
    })
})

In [9]:
dataset1=load_dataset('krinal214/tydiqa_ben_tel')

Downloading:   0%|          | 0.00/948 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/24.2M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/krinal214--tydiqa_ben_tel_v2-6afde2760ba4192b/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
from datasets import concatenate_datasets
datasets['train']=concatenate_datasets([datasets['train'],dataset1['train']])

In [11]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 89881
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5077
    })
})

In [12]:
import re
def filter_dataset(ds,lang):
    def check(d):
        for i in lang:
            if re.search(i,d):
                return True
        return False
    ds = ds.filter(lambda x: check(x['id']))
    return ds

In [13]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [14]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=8):
    picks = random.sample(range(0, len(dataset)-1), num_examples)
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

### Preparing the training and validation dataset

In [15]:
# datasets["train"]=filter_dataset(datasets["validation"],['bengali'])
datasets["validation"]=filter_dataset(datasets["validation"],['bengali', 'telugu'])

  0%|          | 0/6 [00:00<?, ?ba/s]

In [16]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,finnish-5529676475121591091-0,Montgomery Burns,"Charles Montgomery ”Monty” Plantagenet Schicklgruber Burns[1] eli johtaja Burns tai herra Burns (Mr. Burns) on kuvitteellinen henkilö piirrossarjassa Simpsonit. Hän on iäkäs miljardööri, Springfieldin rikkain asukas, Springfieldin ydinvoimalan omistaja, perustaja ja johtaja sekä Homer Simpsonin työnantaja. Burns kuvataan pahuutta huokuvana sydämettömänä ja ahneena riistäjänä, jonka ainoa ystävä on hänen uskollinen henkilökohtainen avustajansa Waylon Smithers. Sekä Burnsin että Smithersin ääninäyttelijä on Harry Shearer, tosin kaikkein ensimmäisissä jaksoissa Burnsia näytteli Christopher Collins eli Chris Latta, joka jäi pian pois Simpsoneista.",Kuka on Mr. Burnsin ääninäyttelijä?,"{'text': ['Harry Shearer'], 'answer_start': [512]}"
1,telugu-1299177877317837031-3226,Adam Vinatieri,"Adam Matthew Vinatieri (born December 28, 1972) is an American football placekicker for the Indianapolis Colts of the National Football League (NFL). He has played in five Super Bowls: four with the New England Patriots and one with the Colts, winning with the Patriots in 2001, 2003, and 2004 and with the Colts in 2006. He holds NFL records for most Super Bowl appearances and wins by a kicker. He also holds NFL records, among all players, for most points scored (2,598)[2][3], most postseason points scored (238)[4], most field goals made (582)[5], and most overtime field goals made (12). He is the only player ever to score 1,000 points with two teams. As of 2018, Vinatieri, 46, is the oldest active player in the NFL and 4th oldest of all time.[6][7]",NFLలో అత్యంత పాత ఆటగాడు ఎవరు?,"{'text': ['Vinatieri'], 'answer_start': [671]}"
2,bengali-1299177877317837031-15138,"King County, Washington","According to the U.S. Census Bureau, the county has a total area of 2,307 square miles (5,980km2), of which 2,116 square miles (5,480km2) is land and 191 square miles (490km2) (8.3%) is water.[16] King County has nearly twice the land area of the state of Rhode Island. The highest point in the county is Mount Daniel at 7,959 feet (2,426 meters) above sea level.","কিং কাউন্টি, ওয়াশিংটন কত বড়?","{'text': ['total area of 2,307 square miles (5,980km'], 'answer_start': [54]}"
3,finnish--8737990199209262431-0,Fenotyyppi,"Fenotyyppi eli yksilön ilmiasu on kehittyvän tai kehittyneen yksilön kaikkien havaittavien ominaisuuksien (anatomisten, fysiologisten, biokemiallisten ja käyttäytymisen) kokonaisuus. Jo 1880-luvulla biologit huomasivat, että perinnöllisyysaine ja eliön ruumis olivat eri asia. Tämä jako selkiytyi, kun varhaiset perinnöllisyystieteilijät ottivat käyttöönsä käsitteet genotyyppi ja fenotyyppi. Fenotyyppi on ympäristön ja geenien vuorovaikutuksen tulos. Näennäisesti samankaltaiset fenotyypit voivat kätkeä geenien tasolla sisäänsä huomattavaa geneettistä muuntelua. Jos ympäristössä ei tapahdu suuria muutoksia, on todennäköisesti fenotyyppikin hyvin samankaltainen kuin aikaisemmissa sukupolvissa ja yksi laji voi kukoistaa miljooniakin vuosia. Jotkut fenotyypin osat voivat kehittyä nopeammin kuin toiset, yksilö ei kehity yhtenä yksikkönä.",Mitä fenotyyppi tarkoittaa?,"{'text': ['kehittyvän tai kehittyneen yksilön kaikkien havaittavien ominaisuuksien (anatomisten, fysiologisten, biokemiallisten ja käyttäytymisen) kokonaisuus'], 'answer_start': [35]}"
4,telugu-1299177877317837031-95,Mahabharata,"Mahabharata (Sanskerta: महाभारत) adalah sebuah karya sastra kuno yang berasal dari India. Secara tradisional, penulis Mahabharata adalah Begawan Byasa atau Vyasa. Buku ini terdiri dari delapan belas kitab, maka dinamakan Astadasaparwa (asta = 8, dasa = 10, parwa = kitab). Namun, ada pula yang meyakini bahwa kisah ini sesungguhnya merupakan kumpulan dari banyak cerita yang semula terpencar-pencar, yang dikumpulkan semenjak abad ke-4 sebelum Masehi.",మహాభారతం యొక్క ఇతిహాసం ఏమిటి?,"{'text': ['karya sastra kuno yang berasal dari India'], 'answer_start': [48]}"
5,english-1038933778432293956-0,

In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

In [18]:
max_length = 384  # The maximum length of a feature (question and context)
doc_stride = 160  # The allowed overlap between two part of the context when splitting is performed.

In [19]:
def prepare_train_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1 :
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1
    
            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (
                offsets[token_start_index][0] <= start_char
                and offsets[token_end_index][1] >= end_char
            ):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while (
                    token_start_index < len(offsets)
                    and offsets[token_start_index][0] <= start_char
                ):
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [20]:
tokenized_datasets = datasets.map(
    prepare_train_features, batched=True, remove_columns=datasets["train"].column_names
)

  0%|          | 0/90 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [21]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-bas

In [22]:
from transformers import TrainingArguments, Trainer
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-model",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    #push_to_hub = True,
    #push_to_hub_model_id = "bert-all-translated",
    save_steps=50000
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/opt/conda/lib/python3.7/site-packages/transformers/training_args.py:951: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case krinal214/bert-all-translated).
  FutureWarning,


In [23]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [24]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning https://huggingface.co/krinal214/bert-all-translated into local empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [25]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 101103
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 930
    })
})


### Training the model

In [26]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 101103
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 6319


Epoch,Training Loss,Validation Loss
1,1.206700,0.577518


***** Running Evaluation *****
  Num examples = 930
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=6319, training_loss=1.4563881737802165, metrics={'train_runtime': 4163.4374, 'train_samples_per_second': 24.284, 'train_steps_per_second': 1.518, 'total_flos': 1.9813414497209856e+16, 'train_loss': 1.4563881737802165, 'epoch': 1.0})

In [27]:
def prepare_validation_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]

    
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

### Evaluating the model

In [28]:
max_answer_length = 50
n_best_answers=20

In [29]:
from tqdm.auto import tqdm
import numpy as np

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 50):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)
    predictions = collections.OrderedDict()
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                        or len(offset_mapping[start_index])==0
                        or len(offset_mapping[end_index])==0
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    try:
                        start_char = offset_mapping[start_index][0]
                        end_char = offset_mapping[end_index][1]
                        valid_answers.append(
                            {
                                "score": start_logits[start_index] + end_logits[end_index],
                                "text": context[start_char: end_char]
                            }
                        )
                    except IndexError:
                        continue

        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}
        
        predictions[example["id"]] = best_answer["text"]
        
    return predictions


In [30]:
import collections
def evaluate(ds,wrong_pred,language):
    val_dataset=filter_dataset(ds,language)
    validation_features = val_dataset.map(
        prepare_validation_features,
        batched=True,
        remove_columns=val_dataset.column_names
    )
    raw_predictions = trainer.predict(validation_features)
    validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))
    
    examples = val_dataset
    features = validation_features

    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)
    final_predictions = postprocess_qa_predictions(val_dataset, validation_features, raw_predictions.predictions,n_best_answers,max_answer_length)
    metric = load_metric("squad")
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in val_dataset]
    for j in range(len(references)):
        wrong=True
        for t in references[j]["answers"]["text"]:
            if normalize_answer(formatted_predictions[j]['prediction_text'])==normalize_answer(t):
                wrong=False
                break
        if wrong:
            wrong_pred.append({"id":references[j]["id"],"prediction":formatted_predictions[j]["prediction_text"],"original:":t})
    return metric.compute(predictions=formatted_predictions, references=references)

### Result

In [31]:
import string
eval_languages = [['bengali'], ['telugu']]
wrong_prediction={}
wrong_prediction['bengali']=[]
wrong_prediction['telugu']=[]
wrong_prediction['all']=[]
for lang in eval_languages:
    output = evaluate(datasets["validation"],wrong_prediction[lang[0]],lang)
    print(lang[0],':',output)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 144
  Batch size = 16


Post-processing 113 example predictions split into 144 features.


  0%|          | 0/113 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 786
  Batch size = 16


Post-processing 669 example predictions split into 786 features.


  0%|          | 0/669 [00:00<?, ?it/s]

telugu : {'exact_match': 69.05829596412556, 'f1': 82.73162103681757}


In [32]:
print('wrong predictions(bengali)',len(wrong_prediction['bengali']))
print('wrong predictions(telugu)',len(wrong_prediction['telugu']))

wrong predictions(bengali) 40
wrong predictions(telugu) 207


In [33]:
output = evaluate(datasets["validation"],wrong_prediction['all'],['telugu','bengali'])
print('overall',output)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 930
  Batch size = 16


Post-processing 782 example predictions split into 930 features.


  0%|          | 0/782 [00:00<?, ?it/s]

overall {'exact_match': 68.41432225063939, 'f1': 81.51556725097583}


### Visualizing wrong predictions

In [37]:
picks = random.sample(range(0, len(wrong_prediction['bengali'])-1), min(40,len(wrong_prediction['bengali'])-1))
for i in picks:
    print(wrong_prediction['bengali'][i])

{'id': 'bengali-94515828498152131-10', 'prediction': '২.৪ মিলিয়ন বছর', 'original:': '৫০০,০০০'}
{'id': 'bengali--2217820890808407965-4', 'prediction': '১৯৮৬ সালের ২৯ এপ্রিল', 'original:': '১৯৮৪ সালের ২৪ অক্টোবর'}
{'id': 'bengali--1366347128155871013-24', 'prediction': 'রাজা বরেণ্য', 'original:': 'কার্তিকেয়'}
{'id': 'bengali-9107229136914417048-1', 'prediction': 'যুক্তরাষ্ট্র', 'original:': 'হাঙ্গেরি'}
{'id': 'bengali--261227273822015974-3', 'prediction': 'যুক্তরাষ্ট্র', 'original:': 'যুক্তরাষ্ট্রের নিউ মেক্সিকো অঙ্গরাজ্যের অ্যামোগোর্দো'}
{'id': 'bengali-5645301196283194243-0', 'prediction': '৬৫০ মিলিয়ন মার্কিন ডলার', 'original:': 'শেখ নাহিয়ানের'}
{'id': 'bengali--9113522782624640859-1', 'prediction': 'চট্টগ্রাম বিশ্ববিদ্যালয়', 'original:': 'চট্টগ্রাম'}
{'id': 'bengali-5988277494911723032-0', 'prediction': '১লা জানুয়ারি, ১৯১৪', 'original:': 'কুমিল্লা জেলার অধীনে ব্রাহ্মণবাড়ীয়া মহকুমার গোকর্ণঘাট'}
{'id': 'bengali--1538257118875639863-2', 'prediction': '১৮৭০ সালে', 'original:': '১৮

In [38]:
picks = random.sample(range(0, len(wrong_prediction['telugu'])-1), min(40,len(wrong_prediction['telugu'])-1))
for i in picks:
    print(wrong_prediction['telugu'][i])

{'id': 'telugu-8243050460142458655-0', 'prediction': '1961 హెక్టార్లలో', 'original:': '1961 హెక్టార్ల'}
{'id': 'telugu--8618704549090519564-0', 'prediction': 'చిన్నూ-సోంబారు', 'original:': 'సోంబారు'}
{'id': 'telugu-1278911153514129585-1', 'prediction': 'గుంటూరు జిల్లా, పెదకూరపాడు మండలం, పాటిబండ్లలో', 'original:': 'గుంటూరు జిల్లా, పెదకూరపాడు మండలం, పాటిబండ్ల'}
{'id': 'telugu-7045541145752173182-0', 'prediction': '302 హెక్టార్లలో', 'original:': '302 హెక్టార్ల'}
{'id': 'telugu--7211034341454150780-1', 'prediction': 'విశాఖపట్నం', 'original:': 'విశాఖపట్నంలో'}
{'id': 'telugu-6739016594433115540-0', 'prediction': 'ఐదుజిల్లాలలో', 'original:': 'ఐదు'}
{'id': 'telugu-8741423891768717983-0', 'prediction': 'D3', 'original:': 'విటమిన్ D3'}
{'id': 'telugu--3775752726340130607-3', 'prediction': 'సుమ', 'original:': 'యేసూ గోఖలే'}
{'id': 'telugu-6643249479996328175-8', 'prediction': 'ఐశ్వర్యా రాయ్', 'original:': 'సెలీన జైట్లీ'}
{'id': 'telugu--4933410501699973162-1', 'prediction': 'తూర్పు గోదావరి జిల్లా 

In [39]:
#trainer.push_to_hub()

Saving model checkpoint to bert-base-multilingual-cased-finetuned-model
Configuration saved in bert-base-multilingual-cased-finetuned-model/config.json
Model weights saved in bert-base-multilingual-cased-finetuned-model/pytorch_model.bin
tokenizer config file saved in bert-base-multilingual-cased-finetuned-model/tokenizer_config.json
Special tokens file saved in bert-base-multilingual-cased-finetuned-model/special_tokens_map.json


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Upload file pytorch_model.bin:   0%|          | 32.0k/676M [00:00<?, ?B/s]

Upload file runs/Apr09_17-19-11_6107c2bb324e/1649524766.2801075/events.out.tfevents.1649524766.6107c2bb324e.35…

Upload file runs/Apr09_17-19-11_6107c2bb324e/events.out.tfevents.1649524766.6107c2bb324e.35.0: 100%|##########…

...

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}}


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

To https://huggingface.co/krinal214/bert-all-translated
   f73364a..c815f6f  main -> main



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'https://huggingface.co/krinal214/bert-all-translated/commit/f73364a66abae55511e894c878c7ea8177a183d0'